In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform

import pandas as pd 
import numpy as np

import glob

In [ ]:
df = pd.read_csv('../input/house-prices-and-images-socal/socal2.csv')
df.head()

In [ ]:
df.shape

In [ ]:
del df['street']
del df['citi']
del df['n_citi']

df.head()

In [ ]:
# list = []
# for i,ids in enumerate(df['image_id']):
#     if i % 1000 == 0:
#         print(i)
#     paths = '../input/house-prices-and-images-socal/socal2/socal_pics/' + str(ids) + '.jpg'
#     image = io.imread(paths)
#     image = image / 255.0
#     image = transform.resize(image,(64,64)).reshape(3,64,64)
#     list.append(ids)

In [ ]:
df = df.sample(256)
df.reset_index(inplace = True )
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
del df['index']
df.head()

In [ ]:
for cols in ['sqft','price']:
    df[cols] = (df[cols].max() - df[cols])/(df[cols].max() - df[cols].min())
    
df.head()

In [ ]:
df.iloc[0,4]

In [ ]:
df.iloc[0,1:4]

In [ ]:
class SocalDataset(Dataset):
    
    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            dataframe: pandas dataframe with features and target.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = '{}{}.jpg'.format(str(self.root_dir), str(self.features.loc[idx,'image_id']))
        image = io.imread(img_name)
        image = image / 255.0
        if len(image.shape) == 3:
            house_features = self.features.iloc[idx, 1:]
            house_features = np.array([house_features]).reshape(4)
            sample = {'image': image, 'house_features': house_features}

            if self.transform:
                sample['image'] = transform.resize(sample['image'],(64,64)).reshape(3,64,64)
                
            sample['image'] = torch.from_numpy(sample['image']).float()
            sample['house_features'] = torch.from_numpy(sample['house_features']).float()

            return sample

In [ ]:
house_dataset = SocalDataset(dataframe=df,
                            root_dir='../input/house-prices-and-images-socal/socal2/socal_pics/',
                            transform = True)

In [ ]:
for i in range(10):
    sample = house_dataset[i]
    print(i, sample['image'].shape, sample['house_features'].shape)

In [ ]:
# lets split the dataset into three parts (train 70%, test 30%)
test_size = 0.3

test_amount = int(house_dataset.__len__() * test_size)

train_set, test_set = torch.utils.data.random_split(house_dataset,[
            (house_dataset.__len__() - test_amount ), test_amount ])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=4,
            shuffle=True,
)

test_dataloader = torch.utils.data.DataLoader(
            test_set,
            batch_size=4,
            shuffle=True,
)

In [ ]:
it = iter(train_dataloader)
items = next(it)
print(type(items))

In [ ]:
print(items['image'].shape)
print(items['house_features'].shape)
print(items['house_features'])

In [ ]:
items['house_features'][:,3]

In [ ]:
items['house_features'][:,:3]

In [ ]:
class TwoInputsNet(nn.Module):
    def __init__(self):
        super(TwoInputsNet, self).__init__()
        self.conv = nn.Conv2d(3,8,kernel_size=3) 
        self.conv1 = nn.Conv2d(8,8,kernel_size=3)
        self.conv2 = nn.Conv2d(8,8,kernel_size=3) 
        self.fc1 = nn.Linear(3,3)
        self.fc2 = nn.Linear(26915,1024)
        self.fc3 = nn.Linear(1024,32) 
        self.fc4 = nn.Linear(32,1) 

    def forward(self, input1, input2):
        c = self.conv(input1)
        c = self.conv1(c)
        c = F.relu(c)
        c = self.conv2(c)
        c = F.relu(c)
        f = self.fc1(input2)
        # now we can reshape `c` and `f` to 2D and concat them
        combined = torch.cat((c.view(c.size(0), -1),
                          f.view(f.size(0), -1)), dim=1)
        out = self.fc2(combined)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)

        
        return out

In [ ]:
network = TwoInputsNet()
print(network)

In [ ]:
items['image'][1]

In [ ]:
items['house_features'][:,:3]

In [ ]:
network.forward(items['image'],items['house_features'][:,:3]).shape

In [ ]:
network.forward(items['image'],items['house_features'][:,:3])

In [ ]:
if torch.cuda.is_available():
    network = network.cuda()

In [ ]:
learning_rate = 0.01
momentum = 0.9
n_epochs = 10

optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)

loss_func = torch.nn.MSELoss()

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=4,
            shuffle=True,
)

test_dataloader = torch.utils.data.DataLoader(
            test_set,
            batch_size=4,
            shuffle=True,
)

In [ ]:
def train(dataloader,epoch):
    batch_idx = 0
    for items in dataloader:
        image = torch.FloatTensor(items['image'])
        features = torch.FloatTensor(items['house_features'][:,:3])
        price = torch.FloatTensor(items['house_features'][:,3])

        if torch.cuda.is_available():
            image = image.cuda()
            features = features.cuda()
            price = price.cuda()
            
        output = network(image,features)
        output = output.reshape(4)
        loss = loss_func(output, price)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
        
        if batch_idx % 4 == 0: #every 25 * batchsize sample we print results
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1, batch_idx * image.shape[0], len(dataloader.dataset),
            100. * batch_idx / len(dataloader), loss.item()))
            
        batch_idx = batch_idx + 1

In [ ]:
for epoch in range(n_epochs):
    # train 
    train(train_dataloader,epoch)